# Analysis of trained U-Net models

In [1]:
import os
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

MODEL_PATH               = "./best_model/CP_epoch7.pth"

TEST_IMAGES_X_PATH       = "./data/ISBI2016_ISIC_Part1_Test_Data/"
TEST_IMAGES_Y_PATH       = "./data/ISBI2016_ISIC_Part1_Test_GroundTruth/"

PREDICTIONS_PATH  = "./analysis/predictions/"

PLOTS_TARGET_PATH        = "./analysis/plots/"

## Helpers

In [2]:
def total_pixels(image):
    width, height = image.size
    return width * height

## Metric Functions

Source: https://en.wikipedia.org/wiki/Sørensen–Dice_coefficient

Sørensen–Dice coefficient is a measure of similarity of 2 images/sets. The set metric is defined as follows:

${\displaystyle DSC={\frac {2|X\cap Y|}{|X|+|Y|}}}$

For boolean data (as in the binary blaack/white segmentation) the following Dice Score is commonly used:

${\displaystyle booleanDSC={\frac {2TP}{2TP+FP+FN}}}$

In [3]:
def DSC(prediction, ground_truth) -> float:
    
    #assert prediction.shape() == ground_truth.shape()
    
    # intersection = 1,1 ; 0,0
    # intersection = |set| - |xor|
    intersection = total_pixels(prediction) - sum(np.logical_xor(prediction, ground_truth))

    numerator   = 2 * intersection 
    denominator = total_pixels(prediction) + total_pixels(ground_truth)

    return numerator / denominator


def booleanDSC(prediction, ground_truth) -> float:
    
    # false_positives = 1,1
    true_positives = sum(np.logical_and(prediction, ground_truth))
    
    # false_positives = 1,0
    # false_negative  = 0,1
    # XOR             = false_positives + false_negatives
    xor = sum(np.logical_xor(prediction, ground_truth))


    numerator   = 2 * true_positives 
    denominator = 2 * true_positives + xor
    
    
    return numerator / denominator

## Running Evaluations

In [34]:
scores_DSC = []
scores_booleanDSC = []

ground_truths = []
predictions = []

for pred_filename in glob.glob(PREDICTIONS_PATH + "*") :

    pred = Image.open(pred_filename)
    predictions.append(pred)

for gt_filename in glob.glob(TEST_IMAGES_Y_PATH + "*") :

    gt = Image.open(gt_filename)
    ground_truths.append(gt)

    


for pred, truth in zip(predictions, ground_truths):
    #pred, truth = pred.view(-1), truth.view(-1)
    truth = truth.getdata()
    print(int(len(truth)))

    print(truth[int(len(truth) / 2) + 1])
    print(pred.size)
    print(truth.size)

    scores_DSC.append(DSC(pred, truth))
    scores_booleanDSC.append(booleanDSC(pred, truth))

786432
0
(430, 322)
(1024, 768)


ValueError: operands could not be broadcast together with shapes (322,430,3) (786432,) 

## Plots and Insights

[1, 2]